<a href="https://colab.research.google.com/github/buseskorkmaz/Sentiment-Analysis-with-Deep-Learning/blob/main/language_transfer_models_XLNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
df = pd.read_json('financial_news_scored.json', orient = 'records', encoding='utf-8')
df['text'] = df['text'].map(lambda x: x.lstrip('BRIEF-'))
df.set_index('index',inplace=True)
df= df.drop(columns=['versionCreated', 'storyId', 'sourceCode', 'storyText'])

In [ ]:
df.head()


,text,score
index,,
1599550751000,Turkish lira edges further into record low ter...,-1
1599476798000,rdogan says Turkish economy will emerge strong...,1
1599249904617,itch Revises 6 Turkish LRG's Outlook to Negati...,-1
1599233579000,Turkey limits maximum period of consumer loans...,-1
1599233165000,Garanti Bankasi Authorizes Head Office To Issu...,1


In [ ]:
df.score.value_counts()

 1    413
-1    254
 0     66
Name: score, dtype: int64

In [ ]:
possible_labels = df.score.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df['label'] = df.score.replace(label_dict)

In [ ]:
import re
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
#df['text'] = df['text'].apply(clean_text)

In [ ]:
df.head()


,text,score,label
index,,,
1599550751000,Turkish lira edges further into record low ter...,-1,0
1599476798000,rdogan says Turkish economy will emerge strong...,1,1
1599249904617,itch Revises 6 Turkish LRG's Outlook to Negati...,-1,0
1599233579000,Turkey limits maximum period of consumer loans...,-1,0
1599233165000,Garanti Bankasi Authorizes Head Office To Issu...,1,1


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.10, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['score', 'label', 'data_type']).count()

text
score label data_type      
-1    0     train       220
            val          34
 0    2     train        56
            val          10
 1    1     train       357
            val          56

In [ ]:
from torch.utils.data import TensorDataset
from transformers import XLNetTokenizer, XLNetModel

In [ ]:
PRE_TRAINED_MODEL_NAME = 'xlnet-large-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=52, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=52, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

633

In [ ]:
len(dataset_val)

100

In [ ]:
from transformers import XLNetForSequenceClassification


In [ ]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(label_dict), mem_len=1024)

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:211: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSeque

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=5e-5, 
                  eps=1e-8)

In [ ]:
epochs = 25

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    overall_acc = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        acc = (len(y_preds[y_preds==label])/len(y_true))*100
        overall_acc += acc * len(y_preds)
        print(f'Accuracy: {acc}\n')

    print(f'Overall Accuracy: {overall_acc/len(dataset_val)}\n')
    

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_XLNET_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.9083465665578843
Validation loss: 0.7216768562793732
F1 Score (Weighted): 0.5833816425120772



Epoch 2
Training loss: 0.6974705681204796
Validation loss: 0.5355025455355644
F1 Score (Weighted): 0.788498780912574



Epoch 3
Training loss: 0.48872329816222193
Validation loss: 0.6358196660876274
F1 Score (Weighted): 0.7660833333333333



Epoch 4
Training loss: 0.2851073831319809
Validation loss: 0.621132418513298
F1 Score (Weighted): 0.7714284745762713



Epoch 5
Training loss: 0.18918049158528447
Validation loss: 1.1207678616046906
F1 Score (Weighted): 0.7847431850789094


...


Epoch 21
Training loss: 0.014195806442876347
Validation loss: 1.6900478452444077
F1 Score (Weighted): 0.8092645447611123



Epoch 22
Training loss: 0.00026015334951807746
Validation loss: 1.7661443799734116
F1 Score (Weighted): 0.7772640641150899



Epoch 23
Training loss: 0.00054458208323922
Validation loss: 1.84768408536911
F1 Score (Weighted): 0.7880727058029691



Epoch 24
Training loss: 0.005317345812363783
Validation loss: 1.8400615751743317
F1 Score (Weighted): 0.7880727058029691



Epoch 25
Training loss: 0.00027251632636762226
Validation loss: 1.8253649175167084
F1 Score (Weighted): 0.7987357252547816



In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: -1
Accuracy: 70.58823529411765

Class: 1
Accuracy: 85.71428571428571

Class: 0
Accuracy: 80.0

Overall Accuracy: 80.0



In [ ]:
f1_score_func(predictions, true_vals)

0.7987357252547816

In [ ]:
model.load_state_dict(torch.load('/content/finetuned_XLNET_epoch_21.model'))
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: -1
Accuracy: 73.52941176470588

Class: 1
Accuracy: 85.71428571428571

Class: 0
Accuracy: 80.0

Overall Accuracy: 81.0



In [ ]:
f1_score_func(predictions, true_vals)

0.8092645447611123